# ONNX Structure Deep Dive: export_modules_as_functions Analysis

This notebook provides a comprehensive examination of ONNX model structures when using different export options, particularly focusing on `export_modules_as_functions` and its implications for hierarchy preservation.

## Table of Contents
1. [Setup and Model Creation](#setup)
2. [Export with Different Options](#export)
3. [Detailed Structure Analysis](#analysis)
4. [Visual Comparison](#visual)
5. [Metadata and Attributes](#metadata)
6. [Implications for Hierarchy Preservation](#implications)
7. [Performance Considerations](#performance)
8. [Conclusions](#conclusions)

## 1. Setup and Model Creation <a name="setup"></a>

First, let's create a test model with clear hierarchical structure to demonstrate the differences.

In [1]:
import torch
import torch.nn as nn
import torch.onnx
import onnx
import onnxruntime as ort
import numpy as np
from pathlib import Path
import json
from typing import Dict, List, Any, Optional
import warnings
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns

# Suppress warnings for clarity
warnings.filterwarnings("ignore", category=UserWarning)

# Set up paths
output_dir = Path("../../temp/onnx_structure_analysis")
output_dir.mkdir(parents=True, exist_ok=True)

print(f"PyTorch version: {torch.__version__}")
print(f"ONNX version: {onnx.__version__}")
print(f"Output directory: {output_dir.absolute()}")

PyTorch version: 2.7.1+cpu
ONNX version: 1.18.0
Output directory: /mnt/d/BYOM/modelexport/notebooks/experimental/../../temp/onnx_structure_analysis


In [ ]:
class ComplexModel(nn.Module):
    """A more complex model to better demonstrate structural differences."""
    
    def __init__(self, input_dim=10, hidden_dim=20, num_classes=5):
        super().__init__()
        
        # Feature extraction layers
        self.feature_extractor = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
        )
        
        # Attention mechanism
        self.attention = nn.MultiheadAttention(
            embed_dim=hidden_dim,
            num_heads=4,
            batch_first=True
        )
        
        # Classification head
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, num_classes)
        )
        
    def forward(self, x):
        # Feature extraction
        features = self.feature_extractor(x)
        
        # Self-attention (reshape for attention layer)
        features_seq = features.unsqueeze(1)  # Add sequence dimension
        attn_out, _ = self.attention(features_seq, features_seq, features_seq)
        attn_out = attn_out.squeeze(1)  # Remove sequence dimension
        
        # Classification
        output = self.classifier(attn_out)
        
        return output

# Create model instance
model = ComplexModel()
model.eval()

# Create sample input
sample_input = torch.randn(2, 10)  # batch_size=2, input_dim=10

print("Model architecture:")
print(model)
print(f"\nTotal parameters: {sum(p.numel() for p in model.parameters()):,}")

## 2. Export with Different Options <a name="export"></a>

Now let's export the model using different configurations of `export_modules_as_functions`.

In [ ]:
def export_model_with_options(model, sample_input, export_modules_as_functions, suffix):
    """Export model with specified options and return path."""
    
    output_path = output_dir / f"model_{suffix}.onnx"
    
    print(f"\nExporting with export_modules_as_functions={export_modules_as_functions}...")
    
    with torch.no_grad():
        torch.onnx.export(
            model,
            sample_input,
            output_path,
            export_params=True,
            opset_version=17,
            do_constant_folding=True,
            input_names=['input'],
            output_names=['output'],
            dynamic_axes={
                'input': {0: 'batch_size'},
                'output': {0: 'batch_size'}
            },
            export_modules_as_functions=export_modules_as_functions,
            verbose=False
        )
    
    print(f"✓ Exported to: {output_path.name}")
    
    # Verify model loads correctly
    try:
        onnx_model = onnx.load(str(output_path))
        onnx.checker.check_model(onnx_model)
        print("✓ Model validation passed")
    except Exception as e:
        print(f"✗ Model validation failed: {e}")
    
    return output_path

# Export with different options
paths = {
    'standard': export_model_with_options(model, sample_input, False, "standard"),
    'all_functions': export_model_with_options(model, sample_input, True, "all_functions"),
}

## 3. Detailed Structure Analysis <a name="analysis"></a>

Let's analyze the internal structure of each exported model in detail.

In [ ]:
def analyze_onnx_structure(onnx_path: Path) -> Dict[str, Any]:
    """Perform comprehensive analysis of ONNX model structure."""
    
    model = onnx.load(str(onnx_path))
    graph = model.graph
    
    analysis = {
        'file_name': onnx_path.name,
        'file_size_mb': onnx_path.stat().st_size / (1024 * 1024),
        'graph': {
            'inputs': len(graph.input),
            'outputs': len(graph.output),
            'nodes': len(graph.node),
            'initializers': len(graph.initializer),
            'value_info': len(graph.value_info)
        },
        'functions': {
            'count': len(model.functions) if hasattr(model, 'functions') else 0,
            'details': []
        },
        'node_types': defaultdict(int),
        'attributes': defaultdict(list),
        'tensor_shapes': {},
        'parameter_count': 0
    }
    
    # Analyze main graph nodes
    for node in graph.node:
        analysis['node_types'][node.op_type] += 1
        
        # Collect attributes
        for attr in node.attribute:
            analysis['attributes'][attr.name].append({
                'node': node.name or node.op_type,
                'type': attr.type
            })
    
    # Analyze functions if present
    if hasattr(model, 'functions') and model.functions:
        for func in model.functions:
            func_info = {
                'name': func.name,
                'domain': func.domain,
                'inputs': len(func.input),
                'outputs': len(func.output),
                'nodes': len(func.node),
                'node_types': defaultdict(int),
                'attributes': len(func.attribute)
            }
            
            # Count node types in function
            for node in func.node:
                func_info['node_types'][node.op_type] += 1
            
            func_info['node_types'] = dict(func_info['node_types'])
            analysis['functions']['details'].append(func_info)
    
    # Count parameters
    for init in graph.initializer:
        shape = [dim for dim in init.dims]
        analysis['parameter_count'] += np.prod(shape) if shape else 1
        analysis['tensor_shapes'][init.name] = shape
    
    # Convert defaultdicts to regular dicts
    analysis['node_types'] = dict(analysis['node_types'])
    analysis['attributes'] = dict(analysis['attributes'])
    
    return analysis

# Analyze all exported models
analyses = {}
for name, path in paths.items():
    print(f"\nAnalyzing {name} export...")
    analyses[name] = analyze_onnx_structure(path)
    print(f"✓ Analysis complete")

In [ ]:
def display_analysis_summary(analyses: Dict[str, Dict[str, Any]]):
    """Display a formatted summary of the analyses."""
    
    print("\n" + "="*80)
    print("ONNX STRUCTURE ANALYSIS SUMMARY")
    print("="*80)
    
    for name, analysis in analyses.items():
        print(f"\n{name.upper()} EXPORT:")
        print("-" * 40)
        
        # Basic info
        print(f"File: {analysis['file_name']}")
        print(f"Size: {analysis['file_size_mb']:.2f} MB")
        
        # Graph structure
        graph = analysis['graph']
        print(f"\nMain Graph:")
        print(f"  Inputs: {graph['inputs']}")
        print(f"  Outputs: {graph['outputs']}")
        print(f"  Nodes: {graph['nodes']}")
        print(f"  Initializers: {graph['initializers']}")
        print(f"  Parameters: {analysis['parameter_count']:,}")
        
        # Node types
        print(f"\nNode Types in Main Graph:")
        for op_type, count in sorted(analysis['node_types'].items()):
            print(f"  {op_type}: {count}")
        
        # Functions
        if analysis['functions']['count'] > 0:
            print(f"\nLocal Functions: {analysis['functions']['count']}")
            for func in analysis['functions']['details'][:5]:  # Show first 5
                print(f"  - {func['name']} (domain: {func['domain']})")
                print(f"    Nodes: {func['nodes']}, I/O: {func['inputs']}/{func['outputs']}")
                print(f"    Operations: {dict(func['node_types'])}")

display_analysis_summary(analyses)

## 4. Visual Comparison <a name="visual"></a>

Let's create visualizations to better understand the structural differences.

In [ ]:
# Create comparison visualizations
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('ONNX Export Structure Comparison', fontsize=16)

# 1. Graph complexity comparison
ax = axes[0, 0]
metrics = ['Nodes', 'Functions', 'Initializers']
standard_values = [
    analyses['standard']['graph']['nodes'],
    analyses['standard']['functions']['count'],
    analyses['standard']['graph']['initializers']
]
functions_values = [
    analyses['all_functions']['graph']['nodes'],
    analyses['all_functions']['functions']['count'],
    analyses['all_functions']['graph']['initializers']
]

x = np.arange(len(metrics))
width = 0.35
ax.bar(x - width/2, standard_values, width, label='Standard Export', alpha=0.8)
ax.bar(x + width/2, functions_values, width, label='Functions Export', alpha=0.8)
ax.set_xlabel('Metric')
ax.set_ylabel('Count')
ax.set_title('Graph Complexity Comparison')
ax.set_xticks(x)
ax.set_xticklabels(metrics)
ax.legend()
ax.grid(True, alpha=0.3)

# 2. Node type distribution - Standard
ax = axes[0, 1]
if analyses['standard']['node_types']:
    node_types = list(analyses['standard']['node_types'].keys())
    node_counts = list(analyses['standard']['node_types'].values())
    ax.pie(node_counts, labels=node_types, autopct='%1.1f%%', startangle=90)
    ax.set_title('Node Types - Standard Export')
else:
    ax.text(0.5, 0.5, 'No nodes in main graph', ha='center', va='center')
    ax.set_title('Node Types - Standard Export')

# 3. Node type distribution - Functions
ax = axes[1, 0]
if analyses['all_functions']['node_types']:
    node_types = list(analyses['all_functions']['node_types'].keys())
    node_counts = list(analyses['all_functions']['node_types'].values())
    ax.pie(node_counts, labels=node_types, autopct='%1.1f%%', startangle=90)
    ax.set_title('Node Types - Functions Export (Main Graph)')
else:
    ax.text(0.5, 0.5, 'Most operations\nin functions', ha='center', va='center', fontsize=12)
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_title('Node Types - Functions Export (Main Graph)')

# 4. File size comparison
ax = axes[1, 1]
export_types = ['Standard', 'Functions']
file_sizes = [
    analyses['standard']['file_size_mb'],
    analyses['all_functions']['file_size_mb']
]
bars = ax.bar(export_types, file_sizes, alpha=0.8, color=['blue', 'orange'])
ax.set_ylabel('File Size (MB)')
ax.set_title('File Size Comparison')
ax.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar, size in zip(bars, file_sizes):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{size:.3f} MB', ha='center', va='bottom')

plt.tight_layout()
plt.show()

## 5. Metadata and Attributes <a name="metadata"></a>

Let's examine the metadata and attributes in both export modes to understand how hierarchy information is preserved.

In [ ]:
def examine_metadata_and_attributes(onnx_path: Path) -> Dict[str, Any]:
    """Examine metadata and attributes in ONNX model."""
    
    model = onnx.load(str(onnx_path))
    
    metadata = {
        'model_metadata': {},
        'graph_metadata': {},
        'node_attributes': defaultdict(list),
        'function_attributes': defaultdict(list)
    }
    
    # Model-level metadata
    if hasattr(model, 'metadata_props'):
        for prop in model.metadata_props:
            metadata['model_metadata'][prop.key] = prop.value
    
    # Graph-level metadata
    if hasattr(model.graph, 'doc_string'):
        metadata['graph_metadata']['doc_string'] = model.graph.doc_string
    
    # Node attributes
    for node in model.graph.node:
        if node.attribute:
            node_info = {
                'node_name': node.name or f"{node.op_type}_unnamed",
                'op_type': node.op_type,
                'attributes': {}
            }
            
            for attr in node.attribute:
                # Extract attribute value based on type
                if attr.type == onnx.AttributeProto.FLOAT:
    .               value = attr.f
                elif attr.type == onnx.AttributeProto.INT:
                    value = attr.i
                elif attr.type == onnx.AttributeProto.STRING:
                    value = attr.s.decode('utf-8') if attr.s else ''
                elif attr.type == onnx.AttributeProto.INTS:
                    value = list(attr.ints)
                elif attr.type == onnx.AttributeProto.STRINGS:
                    value = [s.decode('utf-8') for s in attr.strings]
                else:
                    value = f"Type: {attr.type}"
                
                node_info['attributes'][attr.name] = value
            
            metadata['node_attributes'][node.op_type].append(node_info)
    
    # Function attributes
    if hasattr(model, 'functions'):
        for func in model.functions:
            func_info = {
                'name': func.name,
                'domain': func.domain,
                'attributes': []
            }
            
            if hasattr(func, 'attribute'):
                for attr in func.attribute:
                    func_info['attributes'].append(attr.name)
            
            metadata['function_attributes'][func.domain].append(func_info)
    
    # Convert defaultdicts
    metadata['node_attributes'] = dict(metadata['node_attributes'])
    metadata['function_attributes'] = dict(metadata['function_attributes'])
    
    return metadata

# Examine metadata for both exports
metadata_analysis = {}
for name, path in paths.items():
    print(f"\nExamining metadata for {name} export...")
    metadata_analysis[name] = examine_metadata_and_attributes(path)

# Display interesting findings
print("\n" + "="*60)
print("METADATA AND ATTRIBUTE ANALYSIS")
print("="*60)

for name, metadata in metadata_analysis.items():
    print(f"\n{name.upper()} EXPORT:")
    print("-" * 30)
    
    # Model metadata
    if metadata['model_metadata']:
        print("Model Metadata:")
        for key, value in metadata['model_metadata'].items():
            print(f"  {key}: {value}")
    else:
        print("Model Metadata: None")
    
    # Sample node attributes
    if metadata['node_attributes']:
        print("\nSample Node Attributes:")
        for op_type, nodes in list(metadata['node_attributes'].items())[:2]:
            print(f"  {op_type}:")
            if nodes:
                sample_node = nodes[0]
                for attr_name, attr_value in sample_node['attributes'].items():
                    print(f"    {attr_name}: {attr_value}")
    
    # Function information
    if metadata['function_attributes']:
        print(f"\nFunction Domains: {list(metadata['function_attributes'].keys())}")
        total_functions = sum(len(funcs) for funcs in metadata['function_attributes'].values())
        print(f"Total Functions: {total_functions}")

## 6. Implications for Hierarchy Preservation <a name="implications"></a>

Let's analyze how each export mode affects hierarchy preservation and compare with modelexport's approach.

In [ ]:
def analyze_hierarchy_preservation():
    """Analyze hierarchy preservation capabilities of different approaches."""
    
    print("\n" + "="*80)
    print("HIERARCHY PRESERVATION ANALYSIS")
    print("="*80)
    
    # Define comparison criteria
    criteria = [
        "Operation-level granularity",
        "Module boundary preservation",
        "Direct operation traceability",
        "Metadata flexibility",
        "Runtime compatibility",
        "Debugging capability",
        "Cross-layer analysis",
        "Parameter attribution"
    ]
    
    # Score each approach (1-5 scale)
    scores = {
        "Standard ONNX": [5, 1, 5, 2, 5, 4, 3, 4],
        "export_modules_as_functions": [2, 5, 2, 3, 3, 3, 2, 3],
        "ModelExport HTP": [5, 4, 5, 5, 5, 5, 5, 5]
    }
    
    # Create comparison table
    print("\nComparison Matrix (1=Poor, 5=Excellent):")
    print("-" * 70)
    print(f"{'Criterion':<30} {'Standard':<12} {'Functions':<12} {'HTP':<12}")
    print("-" * 70)
    
    for i, criterion in enumerate(criteria):
        print(f"{criterion:<30} {scores['Standard ONNX'][i]:<12} "
              f"{scores['export_modules_as_functions'][i]:<12} "
              f"{scores['ModelExport HTP'][i]:<12}")
    
    print("-" * 70)
    totals = {name: sum(score) for name, score in scores.items()}
    print(f"{'TOTAL SCORE':<30} {totals['Standard ONNX']:<12} "
          f"{totals['export_modules_as_functions']:<12} "
          f"{totals['ModelExport HTP']:<12}")
    
    # Detailed analysis
    print("\n\nDETAILED ANALYSIS:")
    print("=" * 60)
    
    print("\n1. STANDARD ONNX EXPORT:")
    print("   Pros:")
    print("   • Full operation visibility")
    print("   • Direct debugging access")
    print("   • Maximum runtime compatibility")
    print("   Cons:")
    print("   • No hierarchy information")
    print("   • Lost module boundaries")
    print("   • Difficult to trace operations to source")
    
    print("\n2. EXPORT_MODULES_AS_FUNCTIONS:")
    print("   Pros:")
    print("   • Preserves module boundaries")
    print("   • Clear module-level organization")
    print("   • Good for module replacement")
    print("   Cons:")
    print("   • Operations hidden in functions")
    print("   • Limited debugging access")
    print("   • May have compatibility issues")
    print("   • Deprecated feature")
    
    print("\n3. MODELEXPORT HTP APPROACH:")
    print("   Pros:")
    print("   • Operation-level granularity")
    print("   • Rich metadata tagging")
    print("   • Full debugging capability")
    print("   • Preserves hierarchy information")
    print("   • Universal compatibility")
    print("   Cons:")
    print("   • Requires custom implementation")
    print("   • Additional metadata overhead")

analyze_hierarchy_preservation()

In [ ]:
# Create visual comparison of approaches
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Radar chart for capability comparison
categories = [
    "Op Granularity",
    "Module Boundaries",
    "Traceability",
    "Flexibility",
    "Compatibility",
    "Debugging"
]

# Scores (normalized to 0-1)
standard_scores = [1.0, 0.2, 1.0, 0.4, 1.0, 0.8]
functions_scores = [0.4, 1.0, 0.4, 0.6, 0.6, 0.6]
htp_scores = [1.0, 0.8, 1.0, 1.0, 1.0, 1.0]

# Number of variables
N = len(categories)

# Compute angle for each axis
angles = [n / float(N) * 2 * np.pi for n in range(N)]
angles += angles[:1]

# Initialize the plot
ax1.set_theta_offset(np.pi / 2)
ax1.set_theta_direction(-1)

# Draw one axis per variable and add labels
ax1.set_xticks(angles[:-1])
ax1.set_xticklabels(categories)

# Draw ylabels
ax1.set_rlabel_position(0)
ax1.set_yticks([0.2, 0.4, 0.6, 0.8, 1.0])
ax1.set_yticklabels(["0.2", "0.4", "0.6", "0.8", "1.0"], color="grey", size=7)
ax1.set_ylim(0, 1)

# Plot data
standard_scores += standard_scores[:1]
functions_scores += functions_scores[:1]
htp_scores += htp_scores[:1]

ax1.plot(angles, standard_scores, 'o-', linewidth=2, label='Standard ONNX', color='blue')
ax1.fill(angles, standard_scores, alpha=0.25, color='blue')

ax1.plot(angles, functions_scores, 'o-', linewidth=2, label='export_modules_as_functions', color='orange')
ax1.fill(angles, functions_scores, alpha=0.25, color='orange')

ax1.plot(angles, htp_scores, 'o-', linewidth=2, label='ModelExport HTP', color='green')
ax1.fill(angles, htp_scores, alpha=0.25, color='green')

ax1.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))
ax1.set_title('Capability Comparison', size=14, y=1.08)

# Use case suitability
ax2.set_title('Use Case Suitability', size=14)

use_cases = ['Debugging', 'Analysis', 'Module\nReplacement', 'Custom\nBackends', 'Research']
standard_suit = [4, 3, 1, 3, 2]
functions_suit = [2, 2, 5, 2, 3]
htp_suit = [5, 5, 3, 5, 5]

x = np.arange(len(use_cases))
width = 0.25

ax2.bar(x - width, standard_suit, width, label='Standard', alpha=0.8, color='blue')
ax2.bar(x, functions_suit, width, label='Functions', alpha=0.8, color='orange')
ax2.bar(x + width, htp_suit, width, label='HTP', alpha=0.8, color='green')

ax2.set_ylabel('Suitability Score')
ax2.set_xlabel('Use Case')
ax2.set_xticks(x)
ax2.set_xticklabels(use_cases)
ax2.legend()
ax2.grid(True, alpha=0.3, axis='y')
ax2.set_ylim(0, 6)

plt.tight_layout()
plt.show()

## 7. Performance Considerations <a name="performance"></a>

Let's analyze the performance implications of each approach.

In [ ]:
def benchmark_onnx_models(paths: Dict[str, Path], num_runs: int = 100):
    """Benchmark inference performance of different ONNX models."""
    
    print("\n" + "="*60)
    print("PERFORMANCE BENCHMARKING")
    print("="*60)
    
    results = {}
    
    # Create test input
    test_input = np.random.randn(10, 10).astype(np.float32)  # Larger batch for better timing
    
    for name, path in paths.items():
        print(f"\nBenchmarking {name} export...")
        
        try:
            # Create inference session
            session = ort.InferenceSession(str(path))
            
            # Get input name
            input_name = session.get_inputs()[0].name
            
            # Warmup
            for _ in range(10):
                _ = session.run(None, {input_name: test_input})
            
            # Benchmark
            import time
            times = []
            
            for _ in range(num_runs):
                start = time.perf_counter()
                _ = session.run(None, {input_name: test_input})
                end = time.perf_counter()
                times.append((end - start) * 1000)  # Convert to ms
            
            results[name] = {
                'mean_ms': np.mean(times),
                'std_ms': np.std(times),
                'min_ms': np.min(times),
                'max_ms': np.max(times),
                'median_ms': np.median(times)
            }
            
            print(f"✓ Mean inference time: {results[name]['mean_ms']:.3f} ms")
            
        except Exception as e:
            print(f"✗ Benchmarking failed: {e}")
            results[name] = None
    
    return results

# Run benchmarks
benchmark_results = benchmark_onnx_models(paths)

# Display results
if all(r is not None for r in benchmark_results.values()):
    print("\n" + "-"*60)
    print("PERFORMANCE SUMMARY:")
    print("-"*60)
    print(f"{'Export Type':<20} {'Mean (ms)':<12} {'Std (ms)':<12} {'Min (ms)':<12} {'Max (ms)':<12}")
    print("-"*60)
    
    for name, result in benchmark_results.items():
        if result:
            print(f"{name:<20} {result['mean_ms']:<12.3f} {result['std_ms']:<12.3f} "
                  f"{result['min_ms']:<12.3f} {result['max_ms']:<12.3f}")

## 8. Conclusions <a name="conclusions"></a>

Let's summarize our findings and provide recommendations.

In [ ]:
def generate_conclusions():
    """Generate comprehensive conclusions from the analysis."""
    
    print("\n" + "="*80)
    print("COMPREHENSIVE CONCLUSIONS")
    print("="*80)
    
    print("\n1. STRUCTURAL DIFFERENCES:")
    print("   • Standard Export: Flat graph with all operations visible")
    print("   • Functions Export: Hierarchical structure with module-level functions")
    print("   • Key Finding: Functions preserve module boundaries but hide operations")
    
    print("\n2. GRANULARITY ANALYSIS:")
    print("   • Standard: Operation-level (MatMul, Add, etc.)")
    print("   • Functions: Module-level (Linear, Sequential, etc.)")
    print("   • ModelExport HTP: Operation-level with module metadata")
    
    print("\n3. USE CASE ALIGNMENT:")
    print("   Standard Export Best For:")
    print("   • Maximum compatibility")
    print("   • Direct operation access")
    print("   • Performance-critical applications")
    
    print("\n   export_modules_as_functions Best For:")
    print("   • Module-level manipulation")
    print("   • Preserving logical structure")
    print("   • Model composition workflows")
    
    print("\n   ModelExport HTP Best For:")
    print("   • Debugging and analysis")
    print("   • Custom backend development")
    print("   • Research and experimentation")
    print("   • Fine-grained operation tracking")
    
    print("\n4. KEY INSIGHTS:")
    print("   • export_modules_as_functions operates at different granularity than HTP")
    print("   • Neither standard nor functions export provides operation-level hierarchy")
    print("   • HTP's metadata approach offers best of both worlds")
    print("   • Functions export may have compatibility/deprecation concerns")
    
    print("\n5. RECOMMENDATIONS:")
    print("   ✓ Continue developing ModelExport HTP as primary strategy")
    print("   ✓ Consider export_modules_as_functions for specific use cases only")
    print("   ✓ Focus on operation-level tagging for maximum flexibility")
    print("   ✓ Ensure compatibility with standard ONNX runtime")
    
    print("\n6. FUTURE DIRECTIONS:")
    print("   • Hybrid approach: Combine module functions with operation tags")
    print("   • Enhanced metadata: Include more context in tags")
    print("   • Tool development: Viewers/analyzers for tagged models")
    print("   • Performance optimization: Minimize tagging overhead")

generate_conclusions()

In [ ]:
# Save analysis results
analysis_summary = {
    'timestamp': str(pd.Timestamp.now()),
    'pytorch_version': torch.__version__,
    'onnx_version': onnx.__version__,
    'model_info': {
        'class': model.__class__.__name__,
        'parameters': sum(p.numel() for p in model.parameters())
    },
    'export_analysis': analyses,
    'benchmark_results': benchmark_results,
    'conclusions': {
        'structural_difference': 'Functions export creates hierarchical structure',
        'granularity_difference': 'Module-level vs operation-level',
        'htp_advantage': 'Provides operation-level granularity with hierarchy metadata',
        'recommendation': 'Continue with HTP strategy as primary approach'
    }
}

# Save to JSON
summary_path = output_dir / 'onnx_structure_analysis_summary.json'
with open(summary_path, 'w') as f:
    json.dump(analysis_summary, f, indent=2, default=str)

print(f"\n✅ Analysis complete! Summary saved to: {summary_path}")
print(f"\n🔍 Key Takeaway: export_modules_as_functions and ModelExport HTP serve different purposes")
print(f"   and operate at different granularities. HTP provides the flexibility needed for")
print(f"   operation-level analysis while preserving hierarchy information.")